In [ ]:
import subprocess
import json
import requests

def get_proxy_ips():
    """
    Runs 'terraform output -json' and returns a list of proxy IPs.
    """
    try:
        result = subprocess.run(
            ["terraform", "output", "-json"],
            capture_output=True,
            text=True,
            check=True
        )
        outputs = json.loads(result.stdout)
        proxy_ips = outputs.get("proxy_ips", {}).get("value", [])
        return proxy_ips
    except subprocess.CalledProcessError as e:
        print("Error executing terraform output:", e)
        return []
    except json.JSONDecodeError as e:
        print("Error decoding JSON output:", e)
        return []

def test_proxy(proxy_url):
    """
    Tests the given proxy URL by sending a GET request to http://httpbin.org/ip.
    Returns a tuple (success, response or error message).
    """
    try:
        # Use both http and https proxies; adjust if needed.
        response = requests.get(
            "http://httpbin.org/ip",
            proxies={"http": proxy_url, "https": proxy_url},
            timeout=10
        )
        if response.status_code == 200:
            return True, response.json()
        else:
            return False, f"Status code: {response.status_code}"
    except Exception as e:
        return False, str(e)

if __name__ == "__main__":
    # Set your Squid credentials (should match those defined in your Terraform variables)
    squid_username = "username"  # Replace with your actual username
    squid_password = "password"  # Replace with your actual password
    proxy_port = 3128  # The port Squid is listening on

    # Get the list of proxy IPs from Terraform outputs
    proxies = get_proxy_ips()
    if not proxies:
        print("No proxy IPs found. Ensure that Terraform has output proxies correctly.")
        exit(1)

    print("Testing proxies:")
    for ip in proxies:
        # Build the proxy URL with basic auth: http://username:password@ip:port
        proxy_url = f"http://{squid_username}:{squid_password}@{ip}:{proxy_port}"
        success, info = test_proxy(proxy_url)
        if success:
            print(f"Proxy {ip} is working. Response: {info}")
        else:
            print(f"Proxy {ip} failed. Error: {info}")

Testing proxies:
Proxy 35.238.223.214 is working. Response: {'origin': '89.20.175.53, 35.238.223.214'}
Proxy 34.59.65.223 is working. Response: {'origin': '89.20.175.53, 34.59.65.223'}
Proxy 35.239.76.161 is working. Response: {'origin': '89.20.175.53, 35.239.76.161'}
Proxy 35.232.58.4 is working. Response: {'origin': '89.20.175.53, 35.232.58.4'}
Proxy 34.121.97.34 is working. Response: {'origin': '89.20.175.53, 34.121.97.34'}
Proxy 34.29.252.80 is working. Response: {'origin': '89.20.175.53, 34.29.252.80'}
Proxy 34.55.178.158 is working. Response: {'origin': '89.20.175.53, 34.55.178.158'}
Proxy 34.170.186.168 is working. Response: {'origin': '89.20.175.53, 34.170.186.168'}
